Weather API - OpenMeteo

* Este notebook ha sido creado para añadir información al dataframe obtenido en el paso anterior. La idea es poder determinar el tiempo que hacía en el puerto de llegada del buque. 

In [1]:
pip install openmeteo-requests

  Obtaining dependency information for openmeteo-requests from https://files.pythonhosted.org/packages/50/1f/61d55b3b0970ba3f601d6775439b8aae75bcfee159c9cc9c61cd27bb588a/openmeteo_requests-1.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for openmeteo-sdk>=1.4.0 from https://files.pythonhosted.org/packages/2b/86/962d0422f8e4a1752327952884361dafb22b849df169a8754113a9383b16/openmeteo_sdk-1.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.0.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install retry-requests

  Obtaining dependency information for retry-requests from https://files.pythonhosted.org/packages/b1/f3/8ce908497bebbc2790ef06240a2c0fb28c096abb59062d88f85090464a5f/retry_requests-2.0.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests-cache

  Obtaining dependency information for requests-cache from https://files.pythonhosted.org/packages/bd/c5/c420b54b6c01d004cff9b6f5a8738e9ec8a2561cbaba78667627f1d69c24/requests_cache-1.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for cattrs>=22.2 from https://files.pythonhosted.org/packages/3a/ba/05df14efaa0624fac6b1510e87f5ce446208d2f6ce50270a89b6268aebfe/cattrs-23.1.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/60.1 kB ? eta -:--:--
   ---------------------------------- ----- 51.2/60.1 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 60.1/60.1 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.8 kB ? eta -:--:--
   ---------------------------------------- 50.8/50.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [4]:
#Voy a importar en primer lugar el dataframe
df = pd.read_csv('../data/transit_clean.csv')
df.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0


In [5]:
# Borramos la primera fila del dataframe
df = df.drop(df.index[0])

In [6]:
df.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0
5,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,01/06/2023 05:19,01/06/2023 11:35,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0


In [7]:
# Para pedirle información a la API de clima en un puerto debemos pasarle su latitud y longitud. Vamos a sacar cuantos destinos tenemos:

df.destino.unique()

array(['IBIZA', 'BARCELONA', 'Valencia', 'MAHON, MENORCA', 'SAGUNTO',
       'AJACCIO', 'LA SPEZIA', 'ALICANTE', 'GENOA', 'SALERNO', 'MALAGA',
       'CADIZ', 'MARSEILLE', 'ALTEA', 'PALERMO', 'CAGLIARI', 'ORAN',
       'A Órdenes', 'OLBIA (TERRANOVA)', 'CIVITAVECCHIA', 'ALCANAR',
       'ALCUDIA', 'PIRAEUS', 'ALMERIA', 'PIOMBINO',
       'SARROCH (PORTO FOXI)', 'CARTAGENA', 'CASTELLON', 'TOULON',
       'GIBRALTAR', 'SEVILLA', 'LIVORNO', 'FOS-SUR-MER', 'LISBOA',
       'MALTA (VALETTA)', 'VILLEFRANCHE-SUR-SAONE', 'GANDIA',
       'SAN PEDRO DEL PINATAR', 'NICE', 'SOUTHAMPTON', 'PROPRIANO',
       'MOTRIL', 'PORT EVERGLADES'], dtype=object)